In [635]:
import pyspark
from operator import add
from pyspark import SparkConf
from pyspark.ml.feature import NGram
from pyspark.sql.functions import col,udf
from pyspark.sql import SQLContext
import numpy as np
from pyspark.mllib.linalg.distributed import CoordinateMatrix, MatrixEntry,IndexedRow,IndexedRowMatrix,RowMatrix
from pyspark.mllib.classification import NaiveBayes,NaiveBayesModel
import string
from pyspark.mllib.util import MLUtils
from pyspark.ml.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint

from pyspark.mllib.linalg import Vectors as MLLibVectors
from pyspark.sql.functions import udf
from pyspark.ml.linalg import VectorUDT as VectorUDTML

from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pyspark.sql import Row

as_ml = udf(lambda v: v.asML() if v is not None else None, VectorUDTML())

#sc = pyspark.SparkContext('local[*]',appName="DocClassification")
#sqlc = SQLContext(sc)

In [636]:
strs = sc.parallelize(["This is something I really want to test.",
                       "To test something is something I really want to do.",
                       "I need to do something that I want to do which is to test something.",
                       "Training something is something that I needed to do.",
                       "I trained something and that something was what I needed to do.",
                       "Something I needed to do was to train something.",
                       "Something trained is something that needed to be done.",
                       "If you want to train something something has to be trained.",
                       "Train it like you've never trained it before.",
                       "To test something is something I really want to do.",
                       "I need to do something that I want to do which is to test something.",
                       "Training something is something that I needed to do.",
                       "I trained something and that something was what I needed to do.",
                       "Something I needed to do was to train something.",
                       "Something trained is something that needed to be done.",
                       "If you want to train something something has to be trained.",
                       "Train it like you've never trained it before.",
                       "To test something is something I really want to do.",
                       "I need to do something that I want to do which is to test something.",
                       "Training something is something that I needed to do.",
                       "I trained something and that something was what I needed to do.",
                       "Something I needed to do was to train something.",
                       "Something trained is something that needed to be done.",
                       "If you want to train something something has to be trained.",
                       "Train it like you've never trained it before."])

labs = sc.parallelize([(0,1),(1,0),(2,1),(3,1),(4,0),(5,0),(6,1),
                      (7,1),(8,0),(9,1),(10,1),(11,0),(12,0),(13,1),
                      (14,1),(15,0),(16,1),(17,1),(18,0),(19,0),(20,1)])

In [637]:
strs = strs.zipWithIndex()\
            .map(lambda x: (x[1],x[0]))\
            .mapValues(lambda x: x.split())\

print(strs.collect())



[(0, ['This', 'is', 'something', 'I', 'really', 'want', 'to', 'test.']), (1, ['To', 'test', 'something', 'is', 'something', 'I', 'really', 'want', 'to', 'do.']), (2, ['I', 'need', 'to', 'do', 'something', 'that', 'I', 'want', 'to', 'do', 'which', 'is', 'to', 'test', 'something.']), (3, ['Training', 'something', 'is', 'something', 'that', 'I', 'needed', 'to', 'do.']), (4, ['I', 'trained', 'something', 'and', 'that', 'something', 'was', 'what', 'I', 'needed', 'to', 'do.']), (5, ['Something', 'I', 'needed', 'to', 'do', 'was', 'to', 'train', 'something.']), (6, ['Something', 'trained', 'is', 'something', 'that', 'needed', 'to', 'be', 'done.']), (7, ['If', 'you', 'want', 'to', 'train', 'something', 'something', 'has', 'to', 'be', 'trained.']), (8, ['Train', 'it', 'like', "you've", 'never', 'trained', 'it', 'before.']), (9, ['To', 'test', 'something', 'is', 'something', 'I', 'really', 'want', 'to', 'do.']), (10, ['I', 'need', 'to', 'do', 'something', 'that', 'I', 'want', 'to', 'do', 'which',

In [638]:
grams = strs.map(lambda x: Row(did=x[0],tokens=x[1]))\
            .toDF()

strs = strs.flatMapValues(lambda x: x)

print(strs.collect())



[(0, 'This'), (0, 'is'), (0, 'something'), (0, 'I'), (0, 'really'), (0, 'want'), (0, 'to'), (0, 'test.'), (1, 'To'), (1, 'test'), (1, 'something'), (1, 'is'), (1, 'something'), (1, 'I'), (1, 'really'), (1, 'want'), (1, 'to'), (1, 'do.'), (2, 'I'), (2, 'need'), (2, 'to'), (2, 'do'), (2, 'something'), (2, 'that'), (2, 'I'), (2, 'want'), (2, 'to'), (2, 'do'), (2, 'which'), (2, 'is'), (2, 'to'), (2, 'test'), (2, 'something.'), (3, 'Training'), (3, 'something'), (3, 'is'), (3, 'something'), (3, 'that'), (3, 'I'), (3, 'needed'), (3, 'to'), (3, 'do.'), (4, 'I'), (4, 'trained'), (4, 'something'), (4, 'and'), (4, 'that'), (4, 'something'), (4, 'was'), (4, 'what'), (4, 'I'), (4, 'needed'), (4, 'to'), (4, 'do.'), (5, 'Something'), (5, 'I'), (5, 'needed'), (5, 'to'), (5, 'do'), (5, 'was'), (5, 'to'), (5, 'train'), (5, 'something.'), (6, 'Something'), (6, 'trained'), (6, 'is'), (6, 'something'), (6, 'that'), (6, 'needed'), (6, 'to'), (6, 'be'), (6, 'done.'), (7, 'If'), (7, 'you'), (7, 'want'), (7, 

In [639]:
print("grams:")
grams.show()


grams:
+---+--------------------+
|did|              tokens|
+---+--------------------+
|  0|[This, is, someth...|
|  1|[To, test, someth...|
|  2|[I, need, to, do,...|
|  3|[Training, someth...|
|  4|[I, trained, some...|
|  5|[Something, I, ne...|
|  6|[Something, train...|
|  7|[If, you, want, t...|
|  8|[Train, it, like,...|
|  9|[To, test, someth...|
| 10|[I, need, to, do,...|
| 11|[Training, someth...|
| 12|[I, trained, some...|
| 13|[Something, I, ne...|
| 14|[Something, train...|
| 15|[If, you, want, t...|
| 16|[Train, it, like,...|
| 17|[To, test, someth...|
| 18|[I, need, to, do,...|
| 19|[Training, someth...|
+---+--------------------+
only showing top 20 rows



In [640]:
_2grams = NGram(n=2, inputCol="tokens", outputCol="2Grams")
_3grams = NGram(n=3, inputCol="tokens", outputCol="3Grams")
_4grams = NGram(n=4, inputCol="tokens", outputCol="4Grams")


_2grams = _2grams.transform(grams)
_3grams = _3grams.transform(grams)
_4grams = _4grams.transform(grams)


_2gramRDD = _2grams.select("did","2Grams").rdd.map(tuple).flatMapValues(lambda x: x)
_3gramRDD = _3grams.select("did","3Grams").rdd.map(tuple).flatMapValues(lambda x: x)
_4gramRDD = _4grams.select("did","4Grams").rdd.map(tuple).flatMapValues(lambda x: x)

print(_2gramRDD.collect())
print("-----------------------********************___________________________")
print(_3gramRDD.collect())
print("-----------------------********************___________________________")
print(_4gramRDD.collect())
print("-----------------------********************___________________________")


[(0, 'This is'), (0, 'is something'), (0, 'something I'), (0, 'I really'), (0, 'really want'), (0, 'want to'), (0, 'to test.'), (1, 'To test'), (1, 'test something'), (1, 'something is'), (1, 'is something'), (1, 'something I'), (1, 'I really'), (1, 'really want'), (1, 'want to'), (1, 'to do.'), (2, 'I need'), (2, 'need to'), (2, 'to do'), (2, 'do something'), (2, 'something that'), (2, 'that I'), (2, 'I want'), (2, 'want to'), (2, 'to do'), (2, 'do which'), (2, 'which is'), (2, 'is to'), (2, 'to test'), (2, 'test something.'), (3, 'Training something'), (3, 'something is'), (3, 'is something'), (3, 'something that'), (3, 'that I'), (3, 'I needed'), (3, 'needed to'), (3, 'to do.'), (4, 'I trained'), (4, 'trained something'), (4, 'something and'), (4, 'and that'), (4, 'that something'), (4, 'something was'), (4, 'was what'), (4, 'what I'), (4, 'I needed'), (4, 'needed to'), (4, 'to do.'), (5, 'Something I'), (5, 'I needed'), (5, 'needed to'), (5, 'to do'), (5, 'do was'), (5, 'was to'), 

In [641]:
strs = strs.union(_2gramRDD).union(_3gramRDD).union(_4gramRDD).map(lambda x: (x[1],x[0]))

strs.collect()


[('This', 0),
 ('is', 0),
 ('something', 0),
 ('I', 0),
 ('really', 0),
 ('want', 0),
 ('to', 0),
 ('test.', 0),
 ('To', 1),
 ('test', 1),
 ('something', 1),
 ('is', 1),
 ('something', 1),
 ('I', 1),
 ('really', 1),
 ('want', 1),
 ('to', 1),
 ('do.', 1),
 ('I', 2),
 ('need', 2),
 ('to', 2),
 ('do', 2),
 ('something', 2),
 ('that', 2),
 ('I', 2),
 ('want', 2),
 ('to', 2),
 ('do', 2),
 ('which', 2),
 ('is', 2),
 ('to', 2),
 ('test', 2),
 ('something.', 2),
 ('Training', 3),
 ('something', 3),
 ('is', 3),
 ('something', 3),
 ('that', 3),
 ('I', 3),
 ('needed', 3),
 ('to', 3),
 ('do.', 3),
 ('I', 4),
 ('trained', 4),
 ('something', 4),
 ('and', 4),
 ('that', 4),
 ('something', 4),
 ('was', 4),
 ('what', 4),
 ('I', 4),
 ('needed', 4),
 ('to', 4),
 ('do.', 4),
 ('Something', 5),
 ('I', 5),
 ('needed', 5),
 ('to', 5),
 ('do', 5),
 ('was', 5),
 ('to', 5),
 ('train', 5),
 ('something.', 5),
 ('Something', 6),
 ('trained', 6),
 ('is', 6),
 ('something', 6),
 ('that', 6),
 ('needed', 6),
 ('to', 

In [642]:
vocab = strs.keys()\
            .distinct()\
            .zipWithIndex()
N = vocab.count()
vocab.collect()

[('something', 0),
 ('really', 1),
 ('like', 2),
 ('test something', 3),
 ('test something.', 4),
 ('that something', 5),
 ('was what I', 6),
 ('If you want', 7),
 ('which is to test', 8),
 ('is something that needed', 9),
 ('need', 10),
 ('which', 11),
 ('trained', 12),
 ('train', 13),
 ('you', 14),
 ('is something', 15),
 ('Something I', 16),
 ('never trained', 17),
 ('test something is', 18),
 ('to do something', 19),
 ('do which is', 20),
 ('something and that', 21),
 ('has to be', 22),
 ('trained it before.', 23),
 ('I really want to', 24),
 ('need to do something', 25),
 ('that I want to', 26),
 ('Training something is something', 27),
 ('something and that something', 28),
 ('needed to be done.', 29),
 ('want to train something', 30),
 ('something has to be', 31),
 ('has to be trained.', 32),
 ('needed', 33),
 ('Something', 34),
 ('something is', 35),
 ('to do.', 36),
 ('to be', 37),
 ('really want to', 38),
 ('I want to', 39),
 ('want to do', 40),
 ('is something I really', 41)

In [643]:
strs = strs.join(vocab)\
            .values()\
            .map(lambda x: (x,1))\
            .reduceByKey(add)\
            .map(lambda x: MatrixEntry(x[0][0],x[0][1],float(x[1])))
strs.collect()

[MatrixEntry(10, 0, 1.0),
 MatrixEntry(18, 8, 1.0),
 MatrixEntry(6, 12, 1.0),
 MatrixEntry(5, 33, 1.0),
 MatrixEntry(17, 45, 1.0),
 MatrixEntry(23, 59, 1.0),
 MatrixEntry(15, 83, 1.0),
 MatrixEntry(18, 104, 1.0),
 MatrixEntry(8, 134, 1.0),
 MatrixEntry(14, 148, 1.0),
 MatrixEntry(6, 204, 1.0),
 MatrixEntry(11, 15, 1.0),
 MatrixEntry(18, 40, 1.0),
 MatrixEntry(18, 72, 1.0),
 MatrixEntry(21, 81, 1.0),
 MatrixEntry(11, 111, 1.0),
 MatrixEntry(10, 128, 1.0),
 MatrixEntry(5, 129, 1.0),
 MatrixEntry(11, 175, 1.0),
 MatrixEntry(24, 182, 1.0),
 MatrixEntry(23, 187, 1.0),
 MatrixEntry(19, 0, 2.0),
 MatrixEntry(20, 5, 1.0),
 MatrixEntry(8, 17, 1.0),
 MatrixEntry(4, 21, 1.0),
 MatrixEntry(0, 41, 1.0),
 MatrixEntry(12, 61, 1.0),
 MatrixEntry(22, 107, 1.0),
 MatrixEntry(23, 108, 1.0),
 MatrixEntry(3, 112, 1.0),
 MatrixEntry(14, 163, 1.0),
 MatrixEntry(9, 210, 1.0),
 MatrixEntry(9, 18, 1.0),
 MatrixEntry(1, 42, 1.0),
 MatrixEntry(16, 57, 1.0),
 MatrixEntry(7, 92, 1.0),
 MatrixEntry(4, 117, 1.0),
 Ma

In [644]:
mat = CoordinateMatrix(strs).toIndexedRowMatrix().rows

mat = mat.map(lambda x: (x.index,x.vector))\
         .join(labs)\
         .values()\
         .map(lambda x: LabeledPoint(x[1],x[0]))

In [645]:
training, test = mat.randomSplit([0.6, 0.4])
model = NaiveBayes.train(training, 1.0)
predictionAndLabel = test.map(lambda p: (model.predict(p.features), p.label))
accuracy = 1.0 * predictionAndLabel.filter(lambda pl: pl[0] == pl[1]).count() / test.count()
print('model accuracy {}'.format(accuracy))

model accuracy 0.3333333333333333


In [ ]:
strs = strs.map(lambda x: MatrixEntry(x[0],x[1],float(x[2])))
mat = CoordinateMatrix(strs).toIndexedRowMatrix().rows.toDF(["did","features"]).join(labs,['did'])
mat = mat.drop("did")#.withColumn("features", as_ml("features"))
mat.show()